In [1]:
import pandas as pd

file1 = 'qrels/dev.tsv'
file2 = 'qrels/test.tsv'
file3 = 'qrels/train.tsv'

df1 = pd.read_csv(file1, delimiter='\t')
df2 = pd.read_csv(file2, delimiter='\t')
df3 = pd.read_csv(file3, delimiter='\t')

merged_df = pd.concat([df1, df2, df3])

merged_file = 'merged.tsv'
merged_df.to_csv(merged_file, sep='\t', index=False)

In [6]:
import csv
import json

def export_csv(merge_file, queries_file, corpus_file, output_file):
    # Read merge.tsv file
    merge_data = {}
    with open(merge_file, 'r') as merge:
        merge_reader = csv.reader(merge, delimiter='\t')
        next(merge_reader)  # Skip header row
        for row in merge_reader:
            query_id, corpus_id, *_ = row  # Take only the first two columns
            merge_data[query_id] = corpus_id

    # Read queries.jsonl file
    queries_data = {}
    with open(queries_file, 'r') as queries:
        for line in queries:
            query = json.loads(line)
            queries_data[query['_id']] = query['text']

    # Read corpus.jsonl file
    corpus_data = {}
    with open(corpus_file, 'r') as corpus:
        for line in corpus:
            doc = json.loads(line)
            corpus_data[doc['_id']] = doc['text']

    # Write to output CSV file
    with open(output_file, 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(["Question", "Answer"])

        for query_id, corpus_id in merge_data.items():
            question = queries_data.get(query_id, "")
            answer = corpus_data.get(corpus_id, "")
            writer.writerow([question, answer])

# Usage example
merge_file = "merged.tsv"
queries_file = "queries.jsonl"
corpus_file = "corpus.jsonl"
output_file = "output.csv"

export_csv(merge_file, queries_file, corpus_file, output_file)
